<a href="https://colab.research.google.com/github/Arturchik27/ML/blob/main/%D0%9B%D0%B0%D0%B1%D0%B0%E2%84%962.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.core.pylabtools import figsize
figsize(10, 10) # Зададим размер картинки

# Монтируем гугл-диск
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv("/content/drive/MyDrive/winequality-red.csv", sep=',')
data.info()

In [ ]:
''' Введем дополнительный параметр reviews, показывающий, что вино хорошего качества в случае значения "1"
или вино плохого качества в случае значения "0". Вино будем считать хорошим в случае значения quality > 6.5 '''

def reviews(row):
    if row.quality > 6.5:
        return 1
    else: 
        return 0
data['reviews'] = data.apply(reviews, axis=1)
data['reviews'].value_counts()

0    1382
1     217
Name: reviews, dtype: int64

Разделим данные на тренировочную и валидационную выборки

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('reviews', axis='columns'), data['reviews'], stratify=data['reviews'])

Pipeline - сама обертка над моделью, то есть объект с методами fit и predict. Мы получим этот объект из функции make_pipeline.
MinMaxScaler - класс, который нормализует признаки. Его можно подключить к нашему пайплайну без оборачивания в дополнительные функции.

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

pipeline = make_pipeline(
    
    MinMaxScaler(),                               # Нормализация
    KNeighborsClassifier(n_neighbors=5),          # Классификатор с наилучшими параметрами
)

pipeline

Обучение пайплайна и просмотр его качества

In [ ]:
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)

0.9825

Этап сохранения обученной модели. Подключаем библиотеку pickle.

In [ ]:
import pickle

# нам нужно открыть целевой файл для записи (параметр w) в двоичном виде (параметр b), чтобы записать туда пикл модели
with open("/content/drive/My Drive/Colab Notebooks/Лаба№2.pickle", 'wb') as opened_file:
    pickle.dump(pipeline, opened_file)

Сымитируем получение модели другим человеком - перезапустим ядро ноутбука (все наши текущие переменные сотрутся из памяти) и загрузим готовую модель

In [ ]:
import pandas as pd
import pickle

with open("/content/drive/My Drive/Colab Notebooks/Лаба№2.pickle", 'rb') as f:
    model = pickle.load(f)
    
model

Считаем заново и проверяем результат работы

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/winequality-red.csv", sep=',')

def reviews(row):
    if row.quality > 6.5:
        return 1
    else: 
        return 0
        
data['reviews'] = data.apply(reviews, axis=1)
data['reviews'].value_counts()

X = data.drop('reviews', axis='columns')
y = data['reviews']

model.score(X, y)

0.9862414008755472